# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as plt

# Loading Data

In [ ]:
df=pd.read_csv('/content/metadata.csv')

In [ ]:
df.head()

,type,start_time,ambient_temperature,battery_id,test_id,uid,filename,Capacity,Re,Rct
0,discharge,[2010. 7. 21. 15. 0. ...,4,B0047,0,1,00001.csv,1.6743047446975208,NaN,NaN
1,impedance,[2010. 7. 21. 16. 53. ...,24,B0047,1,2,00002.csv,NaN,0.05605783343888099,0.20097016584458333
2,charge,[2010. 7. 21. 17. 25. ...,4,B0047,2,3,00003.csv,NaN,NaN,NaN
3,impedance,[2010 7 21 20 31 5],24,B0047,3,4,00004.csv,NaN,0.05319185850921101,0.16473399914864734
4,discharge,[2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...,4,B0047,4,5,00005.csv,1.5243662105099023,NaN,NaN


# Generic Dataset Informations
Repeated charge and discharge cycles result in accelerated aging of the batteries while impedance measurements provide insight into the internal battery parameters that change as aging progresses.

**Charge profile:**


* The charge profile for all battery tests seems to be identifical.Charging was carried out in a constant current (CC) mode at 1.5A until the battery voltage reached 4.2V and then continued in a constant voltage (CV) mode until the charge current dropped to 20mA.



**Discharge:**


* Discharge profiles were different from battery to battery.Discharge was carried out at a constant current (CC) level of 1-4 A until the battery voltage fell to values such 2.7V, 2.5V, 2.2V and 2.5V.




**Impedance:**


* Impedance measurement was carried out through an electrochemical impedance spectroscopy (EIS) frequency sweep from     0.1Hz to 5kHz.

The experiments were stopped when the batteries reached a given end-of-life (EOL) criteria: for example 30% fade in rated capacity (from 2Ahr to 1.4Ahr). Other stopping criteria were used such as 20% fade in rated capacity. Note that for batteries 49,50,51,52, the experiments were not stop due to battery EOL but because the software has crashed.



# Working On Data

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   type                 7565 non-null   object
 1   start_time           7565 non-null   object
 2   ambient_temperature  7565 non-null   int64 
 3   battery_id           7565 non-null   object
 4   test_id              7565 non-null   int64 
 5   uid                  7565 non-null   int64 
 6   filename             7565 non-null   object
 7   Capacity             2794 non-null   object
 8   Re                   1956 non-null   object
 9   Rct                  1956 non-null   object
dtypes: int64(3), object(7)
memory usage: 591.1+ KB


In [ ]:
print(df.columns)


Index(['type', 'start_time', 'ambient_temperature', 'battery_id', 'test_id',
       'uid', 'filename', 'Capacity', 'Re', 'Rct'],
      dtype='object')


In [ ]:
print(df.isnull().sum())


type                      0
start_time                0
ambient_temperature       0
battery_id                0
test_id                   0
uid                       0
filename                  0
Capacity               4771
Re                     5609
Rct                    5609
dtype: int64


# Filtering Data

In [ ]:
impedance_data = df[(df['type'] == 'impedance') & df[['Re', 'Rct']].notna().all(axis=1)]

**Sorting Values**

In [ ]:
impedance_data = impedance_data.sort_values(by='start_time')

# Plots

**Electrolyte Resistance (Re) vs Time**

In [ ]:
fig1 = px.line(
    impedance_data, x='start_time', y='Re',
    title='Electrolyte Resistance (Re) vs Time',
    labels={'start_time': 'Time', 'Re': 'Electrolyte Resistance (Ohms)'}
)

fig1.update_layout(
    height=400,
    width=900
)

fig1.show()

**Charge Transfer Resistance (Rct) vs Time**

In [20]:
fig2 = px.line(
    impedance_data, x='start_time', y='Rct',
    title='Charge Transfer Resistance (Rct) vs Time',
    labels={'start_time': 'Time', 'Rct': 'Charge Transfer Resistance (Ohms)'}
)

# Update layout to set a fixed height and width
fig2.update_layout(
    height=600,
    width=900,
)
fig2.show()

**Battery Impedance (Re + Rct) vs Time**

In [ ]:
impedance_data['Battery_Impedance'] = impedance_data['Re'] + impedance_data['Rct']
fig3 = px.line(
    impedance_data, x='start_time', y='Battery_Impedance',
    title='Battery Impedance (Re + Rct) vs Time',
    labels={'start_time': 'Time', 'Battery_Impedance': 'Battery Impedance (Ohms)'}
)
fig3.show()

In [ ]:
fig_combined = plt.Figure()

**Combined Plot**

In [ ]:
fig_combined.add_trace(plt.Scatter(
    x=impedance_data['start_time'], y=impedance_data['Re'],
    mode='lines', name='Electrolyte Resistance (Re)'
))

fig_combined.add_trace(plt.Scatter(
    x=impedance_data['start_time'], y=impedance_data['Rct'],
    mode='lines', name='Charge Transfer Resistance (Rct)'
))

fig_combined.add_trace(plt.Scatter(
    x=impedance_data['start_time'], y=impedance_data['Battery_Impedance'],
    mode='lines', name='Battery Impedance (Re + Rct)'
))

In [ ]:
fig_combined.update_layout(
    title='Battery Parameters Over Time',
    xaxis_title='Time',
    yaxis_title='Values (Ohms)',
    legend=dict(x=0, y=1, traceorder='normal')
)
fig_combined.show()